# Parameter-Efficient Fine-Tuning (PEFT) with NeMo

In this example, we utilize NeMo's [PEFT](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/nemo_megatron/prompt_learning.html)
methods to showcase how to adapt a large language model (LLM) to 
a downstream task, such as financial sentiment predictions. 

With one line configuration change, you can try different PEFT techniques such as [p-tuning](https://arxiv.org/abs/2103.10385), [adapters](https://proceedings.mlr.press/v97/houlsby19a.html), or [LoRA](https://arxiv.org/abs/2106.09685), which add a small number of trainable parameters to the LLM
that condition the model to produce the desired output for the downstream task.

For more details, see the [PEFT script](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/megatron_gpt_peft_tuning.py) in NeMo, which we adapt using NVFlare's Lightning client API to run in a federated scenario.

## Dependencies
We assume you followed the instructions [here](../../README.md#requirements) 
to install the NeMo framework and the NeMo-NVFlare package. 

## Download the pre-trained LLM
In this example, we use a `MegatronGPTModel`, a transformer-based language model based on the GPT architecture.

In [1]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
MegatronGPTModel.list_available_models()

[NeMo W 2023-10-24 20:39:39 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
      from pandas.core.computation.check import NUMEXPR_INSTALLED
    


[PretrainedModelInfo(
 	pretrained_model_name=megatron_gpt_345m,
 	description=345M parameter GPT generative Megatron model.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo
 )]

In [2]:
# Download the model from NGC
import os
model_file = "megatron_gpt_345m.nemo"
if not os.path.isfile(model_file):
    !wget "https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/$model_file"
else:
    print(f"{model_file} already downloaded.")

megatron_gpt_345m.nemo already downloaded.


## Data preprocessing
As our downstream task, we will use the [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for sentiment analysis.

The Financial PhraseBank dataset contains the sentiments for financial news headlines from a retail investor's perspective. Further details about the dataset can be found in Malo et al.'s ["Good Debt or Bad Debt: Detecting Semantic Orientations in Economic Texts"](https://arxiv.org/abs/1307.5336).


#### 1. Download the preprocessing scripts
We use the preprocessing scripts provided by NeMo which can be downloaded from GitHub.

In [3]:
script_name = "prompt_learning_financial_phrase_bank_preprocessing.py"
if not os.path.isfile(script_name):
    !wget -N "https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/nlp/financial_phrase_bank/$script_name"
else:
    print(f"{script_name} already downloaded.")

--2023-10-24 20:40:33--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/nlp/financial_phrase_bank/prompt_learning_financial_phrase_bank_preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4900 (4.8K) [text/plain]
Saving to: ‘prompt_learning_financial_phrase_bank_preprocessing.py’

prompt_learning_fin 100%[===================>]   4.79K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2023-10-24 20:40:33 (32.6 MB/s) - ‘prompt_learning_financial_phrase_bank_preprocessing.py’ saved [4900/4900]



#### 2. Download the Financial PhraseBank Dataset

Download the `FinancialPhraseBank-v1.0.zip` dataset from [here](https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v1.0/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v1.0.zip).

Then extract it under `./data`.

#### 3. Preprocess the dataset

In [4]:
!python3 prompt_learning_financial_phrase_bank_preprocessing.py

Saving train split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl
100%|███████████████████████████████████| 1811/1811 [00:00<00:00, 288663.24it/s]
Saving val split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_val.jsonl
100%|█████████████████████████████████████| 226/226 [00:00<00:00, 294657.35it/s]
Saving test split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_test.jsonl
100%|█████████████████████████████████████| 227/227 [00:00<00:00, 298185.72it/s]


#### 4. Split the dataset to simulate clients
Next, we use three clients to simulate federated learning for p-tuning with NeMo.

In [5]:
!python3 data/split_financial_phrase_data.py --data_path data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl --num_clients 3 --out_dir data/FinancialPhraseBank-v1.0_split

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
Loaded training data with 1811 entries
Save split 1 of 3 with 604 entries to data/FinancialPhraseBank-v1.0_split/site-1.jsonl
Save split 2 of 3 with 604 entries to data/FinancialPhraseBank-v1.0_split/site-2.jsonl
Save split 3 of 3 with 603 entries to data/FinancialPhraseBank-v1.0_split/site-3.jsonl


## Federated learning simulations
Next, we are using NVFlare's [simulator](https://nvflare.readthedocs.io/en/latest/user_guide/fl_simulator.html) to simulate each client training on their own dataset locally and all three clients training together using the [FedAvg](https://arxiv.org/abs/1602.05629) algorithm implemented in NVFlare.

With this setting, we require a GPU with at least 16GB of memory to run all clients in parallel on the same GPU. 
If you have multiple GPUs in your system, you can use the `gpu` argument to assign one GPU for each client, e.g., `gpu="0,1"`.

We will use NVFlare's job command for each setting to create the configurations needed to train the models based on the [sag_nemo](https://github.com/NVIDIA/NVFlare/blob/main/job_templates/sag_pt_deploy_map/info.md) job template. This template allows the definition of different configurations for each client, which we will use to assign their local training data file to each of them.

#### 1. Local P-Tuning
First, we create the job files and modify them to include the data paths for each client and the pre-trained LLM using the `-f` option.
Note, the `app_config` options are specific to the app script (`megatron_gpt_peft_tuning.py`) and modify variables in the NeMo config file (`megatron_gpt_peft_tuning_config.yaml`) directly on execution.

At this point, we also modify the local number of clients, local epochs and FL rounds to simulate local training.

The PEFT method is "ptuning".

In [20]:
%env NVFLARE_HOME=/home/hroth/Code2/nvflare/nemo_peft_example
#!python3 -m pip install -e /home/hroth/Code2/nvflare/nemo_peft_example

import os
peft_scheme="model.peft.peft_scheme\=ptuning" # can be either ptuning, adapter, lora, or ia3
app_script="megatron_gpt_peft_tuning.py"
restore_from_path=f"model.restore_from_path\={os. getcwd()}/megatron_gpt_345m.nemo"
trainer_config="trainer.max_steps\=10000 trainer.val_check_interval\=100"
val_files=f"model.data.validation_ds.file_names\=\[{os. getcwd()}/data/FinancialPhraseBank-v1.0/financial_phrase_bank_val.jsonl\]"
train_files_prefix=f"model.data.train_ds.file_names\=\[{os. getcwd()}/data/FinancialPhraseBank-v1.0_split/site"

!nvflare job create -force -j "./jobs/peft_p-tuning_local_345M" -w "sag_nemo" -sd "code" \
   -f app_1/config_fed_client.conf app_script={app_script} app_config="{peft_scheme} {restore_from_path} {trainer_config} {val_files} {train_files_prefix}-1.jsonl\]" \
   -f app_2/config_fed_client.conf app_script={app_script} app_config="{peft_scheme} {restore_from_path} {trainer_config} {val_files} {train_files_prefix}-2.jsonl\]" \
   -f app_3/config_fed_client.conf app_script={app_script} app_config="{peft_scheme} {restore_from_path} {trainer_config} {val_files} {train_files_prefix}-3.jsonl\]" \
   -f app_server/config_fed_server.conf num_rounds=1

env: NVFLARE_HOME=/home/hroth/Code2/nvflare/nemo_peft_example

The following are the variables you can change in the template

---------------------------------------------------------------------------------------------------------------------------------------
                                                                                                                                       
  job folder: ./jobs/peft_p-tuning_local_345M                                                                                            
                                                                                                                                       
---------------------------------------------------------------------------------------------------------------------------------------
  file_name                      var_name                       value                               component                          
-------------------------------------------------------

Next, simulate each client p-tuning on their local dataset using the FL simulator. To do this, we only run 1 round of FL, with each client running 50 p-tuning epochs on their local dataset.

In [ ]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/peft_p-tuning_local_345M",
    workspace="/tmp/nvflare/nemo/peft_p-tuning_local_345M",
    n_clients=3,
    threads=3
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-10-25 15:13:58,822 - SimulatorRunner - INFO - Create the Simulator Server.
2023-10-25 15:13:58,825 - CoreCell - INFO - server: creating listener on tcp://0:38913
2023-10-25 15:13:58,846 - CoreCell - INFO - server: created backbone external listener for tcp://0:38913
2023-10-25 15:13:58,848 - ConnectorManager - INFO - 15263: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-10-25 15:13:58,850 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:50345] is starting
2023-10-25 15:13:59,352 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:50345
2023-10-25 15:13:59,354 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:38913] is starting
2023-10-25 15:13:59,434 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 51887
2023-10-25 15:13:59,435 - SimulatorRunner - INFO - Deploy the Apps.
2023-10-25 15:13:59,444 - SimulatorRunner - INFO - Create

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


NEMO version 1.21.0rc0
NEMO version 1.21.0rc0
2023-10-25 15:14:09,207 - IntimeModelSelector - INFO - model selection weights control: None
2023-10-25 15:14:09,208 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: Server runner starting ...
[NeMo I 2023-10-25 15:14:09 megatron_trainer_builder:49] Detected interactive environment, using NLPDDPStrategyNotebook


[NeMo W 2023-10-25 15:14:09 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
      rank_zero_warn(
    


2023-10-25 15:14:09,484 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: True (cuda), used: True
2023-10-25 15:14:09,518 - SimulatorClientRunner - INFO - Start the clients run simulation.
2023-10-25 15:14:09,520 - pytorch_lightning.utilities.rank_zero - INFO - TPU available: False, using: 0 TPU cores
2023-10-25 15:14:09,522 - pytorch_lightning.utilities.rank_zero - INFO - IPU available: False, using: 0 IPUs
2023-10-25 15:14:09,523 - pytorch_lightning.utilities.rank_zero - INFO - HPU available: False, using: 0 HPUs
2023-10-25 15:14:10,523 - SimulatorClientRunner - INFO - Simulate Run client: site-1 on GPU group: None
2023-10-25 15:14:10,524 - SimulatorClientRunner - INFO - Simulate Run client: site-2 on GPU group: None
2023-10-25 15:14:10,544 - SimulatorClientRunner - INFO - Simulate Run client: site-3 on GPU group: None
2023-10-25 15:14:11,655 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connection [CN00006 127.0.0.1:38913 <= 127.0.0.1:56566] is created: PID: 15263
2023-1

...

[NeMo I 2023-10-25 15:14:14 nlp_overrides:686] Model MegatronGPTSFTModel was successfully restored from /home/hroth/Code2/nvflare/nemo_peft_example/integration/nemo/examples/peft/megatron_gpt_345m.nemo.
2023-10-25 15:14:14,363 - root - INFO - Adding adapter weights to the model for PEFT
[NeMo I 2023-10-25 15:14:14 nlp_adapter_mixins:182] Before adding PEFT params:
      | Name        | Type       | Params
    -------------------------------------------
    0 | model       | GPTModel   | 354 M 
    1 | val_metric  | ModuleList | 0     
    2 | test_metric | ModuleList | 0     
    -------------------------------------------
    0         Trainable params
    354 M     Non-trainable params
    354 M     Total params
    1,419.485 Total estimated model params size (MB)
[NeMo I 2023-10-25 15:14:14 nlp_adapter_mixins:195] After adding PEFT params:
      | Name        | Type       | Params
    -------------------------------------------
    0 | model       | GPTModel   | 356 M 
    1 | val_m

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


2023-10-25 15:14:14,870 - numexpr.utils - INFO - Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-10-25 15:14:14,870 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


2023-10-25 15:14:15,693 - Cell - INFO - Register blob CB for channel='aux_communication', topic='*'
2023-10-25 15:14:15,836 - Cell - INFO - Register blob CB for channel='aux_communication', topic='*'
2023-10-25 15:14:15,923 - Cell - INFO - Register blob CB for channel='aux_communication', topic='*'
2023-10-25 15:14:16,214 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_and_gather, peer=site-2, peer_run=simulate_job, task_name=train, task_id=4ea09bb0-13c6-40b6-81a6-04870005360b]: assigned task to client site-2: name=train, id=4ea09bb0-13c6-40b6-81a6-04870005360b
2023-10-25 15:14:16,217 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_and_gather, peer=site-2, peer_run=simulate_job, task_name=train, task_id=4ea09bb0-13c6-40b6-81a6-04870005360b]: sent task assignment to client. client_name:site-2 task_id:4ea09bb0-13c6-40b6-81a6-04870005360b
2023-10-25 15:14:16,218 - GetTaskCommand - INFO - return task to client.  client_name:

#### 2. Federated P-Tuning
We use the [FedAvg](https://arxiv.org/abs/1602.05629) algorithm to p-tune the model in a federated scenario. First, create and modify the configuration files again. 
This time, we increase the number of FL rounds and decrease the number of local epochs per round to match the federated scenario.

In [ ]:
!python3 create_configs.py --job_folder "jobs/peft_p-tuning_fedavg_345M" --num_clients 3 --max_steps 200 --num_rounds 50

Next, simulate the federated p-tuning using FedAvg. Here, each client p-tunes for one local epoch before sending their local model updates to the server for aggregation. This is repeated for 50 FL rounds.

In [ ]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/peft_p-tuning_fedavg_345M",
    workspace="/tmp/nvflare/nemo/peft_p-tuning_fedavg_345M",
    n_clients=3,
    threads=3
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

You can visualize the training process using TensorBoard

In [ ]:
!tensorboard --logdir /tmp/nvflare/nemo

## Results
In this scenario, all clients utilize the same validation set, allowing for a direct comparison between the locally p-tuned and federated global models. As anticipated, the FedAvg-trained global model exhibits lower validation loss than the models trained solely on their local datasets. This is because the global model has access to all client datasets and can, consequently, generalize better.

![validation loss](./figs/val_loss.svg)

## Inference

We can use `model.generate()` to run inference after p-tuning the model. 
Let's define some test examples to feed to the p-tuned model to see its predictions.

In [ ]:
test_examples = [
    {"taskname": "sentiment", "sentence": "The products have a low salt and fat content ."},
    {"taskname": "sentiment", "sentence": "The agreement is valid for four years ."},
    {"taskname": "sentiment", "sentence": "Diluted EPS rose to EUR3 .68 from EUR0 .50 ."},
    {"taskname": "sentiment", "sentence": "The company is well positioned in Brazil and Uruguay ."},
    {"taskname": "sentiment", "sentence": "Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier ."},
]

Next, we will load the global model.

In [ ]:
import os
import torch
import pytorch_lightning as pl
from nemo_nvflare.fed_megatron_gpt_prompt_learning_model import FedMegatronGPTPromptLearningModel
from nemo_nvflare.utils import load_weights
from omegaconf import OmegaConf
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPStrategy
from pytorch_lightning.plugins.environments import TorchElasticEnvironment

# Load model configuration used by one of the clients
config = OmegaConf.load("jobs/gpt_p-tuning_fedavg_345M/server/config/megatron_gpt_prompt_learning_config.yaml")

# Set GPT model path
config.model.language_model_path = "megatron_gpt_345m.nemo"

# Load task templates
config.model.task_templates = OmegaConf.load("jobs/gpt_p-tuning_fedavg_345M/server/config/task_templates.json")

# Set task that were learned
config.model.new_tasks = ["sentiment"]

# Setup cluster environment parameters
# use torch elastic cluster environment so `create_process_externally` is True
# the launcher is set to None. It will not try to spawn new processes.
# It won't create the misconfiguration error because of the `interactive session`
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'
strategy = NLPDDPStrategy(find_unused_parameters=False, no_ddp_communication_hook=True)
plugins = [TorchElasticEnvironment()]

# Set up the trainer and load the model that was used for p-tuning
trainer = pl.Trainer(plugins=plugins, strategy=strategy, **config.trainer)
model = FedMegatronGPTPromptLearningModel(cfg=config.model, trainer=trainer)
model.init_prompt_encoder()

print("Model initialized", type(model))

Overwrite the prompt encoder with the best global model

In [ ]:
ckpt = torch.load("/tmp/nvflare/nemo/gpt_p-tuning_fedavg_345M/simulate_job/app_server/best_FL_global_model.pt")
global_weights = ckpt["model"]

n_loaded = load_weights(model, global_weights, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
print(f"Loaded {n_loaded} of {len(global_weights)} weights")

Run the model

In [ ]:
response = model.generate(inputs=test_examples, length_params=None)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print(result)
    print("-" * 30)

The expected output predictions look something like this

>      The products have a low salt and fat content . sentiment: neutral
>      ------------------------------
>      The agreement is valid for four years . sentiment: neutral
>      ------------------------------
>      Diluted EPS rose to EUR3 .68 from EUR0 .50 . sentiment: positive
>      ------------------------------
>      The company is well positioned in Brazil and Uruguay . sentiment: positive
>      ------------------------------
>      Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier . sentiment: negative
>      ------------------------------